In [13]:
import pandas as pd
import os
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import numpy as np
import os

import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# Read Data

In [14]:
def process_pea_embryo_data(file_name):
    def merge_columns(primary, secondary):
        return primary.fillna(secondary)
    
    # Load the data with the correct header
    df = pd.read_excel(file_name, header=2)

    # Filter rows based on Identification Level
    df = df[df['Unnamed: 9'].isin(['Tier 1', 'Tier 2'])]

    # Drop the unnecessary columns
    columns_to_drop = ["No", "NovaMT Library No..1", "Mass Error (ppm).1", "RT Error (s).1", 
                       "Zero-reaction", "One-reaction", "Two-reaction", "Unnamed: 23", 
                       "m/z_heavy", "Intensity", "nCharge", 
                       "nTag", "NovaMT Library No.", "Mass Error (ppm)", "RT Error (s)", 
                       "NovaMT Library No.", "Mass Error (ppm)", "RT Error (s)", "NovaMT Library No."]
    df['Compound'] = merge_columns(df['Compound'], df['Compound.1'])
    df['External Identifier'] = merge_columns(df['External Identifier'], df['External Identifier.1'])

    # Drop the redundant columns
    df = df.drop(columns=columns_to_drop)
    df = df.drop(columns=['Compound.1', 'External Identifier.1'])

    # Rename columns
    df = df.rename(columns={'Unnamed: 9': 'Identification Level'})
    df = df.rename(columns={'External Identifier': 'External Identifier'})

    # Sort by External Identifier
    df = df.sort_values(by='External Identifier')

    return df

In [15]:
def compare_groups(data, group1_prefix, group2_prefix):
    # Filter columns for each group
    group1_columns = [col for col in data.columns if col.startswith(group1_prefix)]
    group2_columns = [col for col in data.columns if col.startswith(group2_prefix)]
    
    # Calculate mean for each group
    data[f'{group1_prefix}_mean'] = data[group1_columns].mean(axis=1)
    data[f'{group2_prefix}_mean'] = data[group2_columns].mean(axis=1)
    
    # Calculate fold change
    data['Fold_Change'] = data[f'{group1_prefix}_mean'] / data[f'{group2_prefix}_mean']
    
    # Number of features (rows)
    number_features = data.shape[0]
    
    # Initialize lists for t-scores, p-values, and FDR values
    t_scores, p_values = [], []
    
    # Perform t-test for each feature
    for ii in range(number_features):
        # Extract the values for the current feature
        group1_values = data[group1_columns].iloc[ii].values
        group2_values = data[group2_columns].iloc[ii].values
        
        # Perform the t-test
        t, p = ttest_ind(group1_values, group2_values, equal_var=False)
        
        # Handle NaN values
        if np.isnan(t):
            t, p = 0, 1
        
        t_scores.append(t)
        p_values.append(p)
    
    # Store t-scores and p-values in the DataFrame
    data['t_score'] = t_scores
    data['p_value'] = p_values
    
    # Apply FDR correction
    fdrs = multipletests(p_values, method='fdr_bh')[1]
    data['FDR_P_Value'] = fdrs
    
    # Rename the columns
    data.rename(columns={'Neutral Mass (Da)': 'm/z', 'RT (s)': 'retention_time'}, inplace=True)
    
    # Sort and keep necessary columns
    necessary_columns = ['m/z', 'retention_time', 'p_value', 't_score']
    final_data = data[necessary_columns]
    
    return final_data

In [16]:
df = process_pea_embryo_data('/Users/bowen/Desktop/multi_omics_Analysis/2408_pea 2/PeaEmbryo.xlsx')
# df2 = process_pea_embryo_data('/Users/bowen/Desktop/multi_omics_Analysis/2408_pea 2/PeaEndosperm.xlsx')
# df = pd.merge(df1, df2, on=['Neutral Mass (Da)'], how='outer')
df = df.dropna(thresh=len(df.columns)-5)
df.rename(columns={"RT (s)_x": "retention_time", "Neutral Mass (Da)": "m/z"}, inplace=True)

In [17]:
df

,RT (s),Normalized RT (s),m/z_light,m/z,Identification Level,External Identifier,Compound,sFED1,sFED1.1,sFED1.2,...,sMED2.2,sMED3,sMED3.1,sMED3.2,sMED4,sMED4.1,sMED4.2,QC,QC.1,QC.2
493,307.728,247.820,481.0835,247.0252,Tier 2,C00018,Pyridoxal 5-phosphate,0.3990,0.3280,0.554,...,1.457,1.1950,1.1700,0.8890,0.6170,0.7250,0.7910,1.0600,1.0130,0.9680
786,368.504,305.333,426.1190,384.1214,Tier 2,C00021,S-Adenosyl-L-homocysteine,0.0790,0.0380,0.438,...,0.220,0.7040,0.1330,0.3140,0.6270,0.6300,0.9410,0.9280,0.9160,0.8780
1053,427.875,359.209,426.1226,384.1286,Tier 2,C00021,Isomer 1 of S-Adenosyl-L-homocysteine,0.6310,0.2888,0.797,...,0.560,1.1870,1.0870,0.9960,1.5980,1.1120,1.2450,0.9940,1.1540,0.9000
183,220.128,155.152,381.1110,147.0527,Tier 1,C00025,Glutamic acid,0.4190,0.5340,0.380,...,0.828,0.9380,0.4550,0.4620,0.5050,1.0520,0.8390,0.9460,0.9550,0.9720
1645,237.245,176.367,280.1180,118.0267,Tier 2,C00042,Isomer 1 of Succinic acid,0.3940,0.8490,0.208,...,0.437,0.9380,0.6750,0.6590,0.8060,1.2350,1.1100,0.9620,0.9550,0.9820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2670,795.500,706.870,442.3331,280.2402,Tier 1,NaN,Linoleic Acid,0.2610,0.5700,0.198,...,0.246,0.0372,0.0381,0.3260,0.0428,0.0330,0.0357,0.4840,0.4280,0.6320
2674,818.200,727.040,466.3299,304.2402,Tier 2,NaN,Arachidonic acid,0.4580,0.6460,0.397,...,0.484,0.3790,0.4640,0.3580,0.3888,0.2994,0.3243,0.4374,0.6420,0.6420
2675,862.000,765.960,468.3446,306.2559,Tier 2,NaN,Icosatrienoic acid,0.9880,0.9250,0.810,...,0.987,0.7530,0.8940,0.7940,0.3406,0.7570,0.2841,0.8610,0.7480,0.3223
2676,868.500,771.730,418.3315,256.2402,Tier 2,NaN,Hexadecanoic acid,1.0680,0.5140,0.950,...,1.028,0.8740,1.0160,0.4010,0.8500,0.8130,0.8890,0.8780,0.8820,0.9920


# TS in sFE

In [5]:
sFED2_vs_sFED1 = compare_groups(df, 'sFED2', 'sFED1')
sFED3_vs_sFED2 = compare_groups(df, 'sFED3', 'sFED2')
sFED4_vs_sFED3 = compare_groups(df, 'sFED4', 'sFED3')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'TS_sFE'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sFED2_vs_sFED1.txt'
file_path = os.path.join(folder_path, file_name)
sFED2_vs_sFED1 .to_csv(file_path, sep='\t', index=False)

file_name = 'sFED3_vs_sFED2.txt'
file_path = os.path.join(folder_path, file_name)
sFED3_vs_sFED2.to_csv(file_path, sep='\t', index=False)

file_name = 'sFED4_vs_sFED3.txt'
file_path = os.path.join(folder_path, file_name)
sFED4_vs_sFED3.to_csv(file_path, sep='\t', index=False)

# TS in ME

In [18]:
sMED2_vs_sMED1 = compare_groups(df, 'sMED2', 'sMED1')
sMED3_vs_sMED2 = compare_groups(df, 'sMED3', 'sMED2')
sMED4_vs_sMED3 = compare_groups(df, 'sMED4', 'sMED3')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'TS_sME'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sMED2_vs_sMED1.txt'
file_path = os.path.join(folder_path, file_name)
sMED2_vs_sMED1.to_csv(file_path, sep='\t', index=False)

file_name = 'sMED3_vs_sMED2.txt'
file_path = os.path.join(folder_path, file_name)
sMED3_vs_sMED2.to_csv(file_path, sep='\t', index=False)

file_name = 'sMED4_vs_sMED3.txt'
file_path = os.path.join(folder_path, file_name)
sMED4_vs_sMED3.to_csv(file_path, sep='\t', index=False)

# FE vs ME

In [7]:
sFED1_vs_sMED1 = compare_groups(df, 'sFED1', 'sMED1')
sFED2_vs_sMED2 = compare_groups(df, 'sFED2', 'sMED2')
sFED3_vs_sMED3 = compare_groups(df, 'sFED3', 'sMED3')
sFED4_vs_sMED4 = compare_groups(df, 'sFED4', 'sMED4')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'sFE_vs_sME'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sFED1_vs_sMED1.txt'
file_path = os.path.join(folder_path, file_name)
sFED1_vs_sMED1 .to_csv(file_path, sep='\t', index=False)

file_name = 'sFED2_vs_sMED2.txt'
file_path = os.path.join(folder_path, file_name)
sFED2_vs_sMED2 .to_csv(file_path, sep='\t', index=False)

file_name = 'sFED3_vs_sMED3.txt'
file_path = os.path.join(folder_path, file_name)
sFED3_vs_sMED3 .to_csv(file_path, sep='\t', index=False)

file_name = 'sFED4_vs_sMED4.txt'
file_path = os.path.join(folder_path, file_name)
sFED4_vs_sMED4 .to_csv(file_path, sep='\t', index=False)

# FEn vs MEn

In [5]:
sMEnD1_vs_sFEnD1 = compare_groups(df, 'sMEnD1', 'sFEnD1')
sMEnD2_vs_sFEnD2 = compare_groups(df, 'sMEnD2', 'sFEnD2')
sMEnD3_vs_sFEnD3 = compare_groups(df, 'sMEnD3', 'sFEnD3')
sMEnD4_vs_sFEnD4 = compare_groups(df, 'sMEnD4', 'sFEnD4')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'sMEn_vs_sFEn'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sMEnD1_vs_sFEnD1.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD1_vs_sFEnD1.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD2_vs_sFEnD2.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD2_vs_sFEnD2.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD3_vs_sFEnD3.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD3_vs_sFEnD3.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD4_vs_sFEnD4.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD4_vs_sFEnD4.to_csv(file_path, sep='\t', index=False)


# TS in FEn

In [6]:
sFEn2_vs_sFEn1 = compare_groups(df, 'sFEnD2', 'sFEnD1')
sFEn3_vs_sFEn2 = compare_groups(df, 'sFEnD3', 'sFEnD2')
sFEn4_vs_sFEn3 = compare_groups(df, 'sFEnD4', 'sFEnD3')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'TS_sFEn'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sFEnD2_vs_sFEnD1.txt'
file_path = os.path.join(folder_path, file_name)
sFEn2_vs_sFEn1.to_csv(file_path, sep='\t', index=False)

file_name = 'sFEnD3_vs_sFEnD2.txt'
file_path = os.path.join(folder_path, file_name)
sFEn3_vs_sFEn2.to_csv(file_path, sep='\t', index=False)

file_name = 'sFEnD4_vs_sFEnD3.txt'
file_path = os.path.join(folder_path, file_name)
sFEn4_vs_sFEn3.to_csv(file_path, sep='\t', index=False)

# TS MEn

In [7]:
sMEn2_vs_sMEn1 = compare_groups(df, 'sMEnD2', 'sMEnD1')
sMEn3_vs_sMEn2 = compare_groups(df, 'sMEnD3', 'sMEnD2')
sMEn4_vs_sMEn3 = compare_groups(df, 'sMEnD4', 'sMEnD3')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'TS_sMEn'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sMEnD2_vs_sMEnD1.txt'
file_path = os.path.join(folder_path, file_name)
sMEn2_vs_sMEn1.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD3_vs_sMEnD2.txt'
file_path = os.path.join(folder_path, file_name)
sMEn3_vs_sMEn2.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD4_vs_sMEnD3.txt'
file_path = os.path.join(folder_path, file_name)
sMEn4_vs_sMEn3.to_csv(file_path, sep='\t', index=False)

# FE vs FEn

In [6]:
sFED1_vs_sFEnD1 = compare_groups(df, 'sFED1', 'sFEnD1')
sFED2_vs_sFEnD2 = compare_groups(df, 'sFED2', 'sFEnD2')
sFED3_vs_sFEnD3 = compare_groups(df, 'sFED3', 'sFEnD3')
sFED4_vs_sFEnD4 = compare_groups(df, 'sFED4', 'sFEnD4')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'sFE_vs_sFEn'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sFED1_vs_sFEnD1.txt'
file_path = os.path.join(folder_path, file_name)
sFED1_vs_sFEnD1.to_csv(file_path, sep='\t', index=False)

file_name = 'sFED2_vs_sFEnD2.txt'
file_path = os.path.join(folder_path, file_name)
sFED2_vs_sFEnD2.to_csv(file_path, sep='\t', index=False)

file_name = 'sFED3_vs_sFEnD3.txt'
file_path = os.path.join(folder_path, file_name)
sFED3_vs_sFEnD3.to_csv(file_path, sep='\t', index=False)

file_name = 'sFED4_vs_sFEnD4.txt'
file_path = os.path.join(folder_path, file_name)
sFED4_vs_sFEnD4.to_csv(file_path, sep='\t', index=False)

# ME vs MEn

In [7]:
sMEnD1_vs_sMED1 = compare_groups(df, 'sMEnD1', 'sMED1')
sMEnD2_vs_sMED2 = compare_groups(df, 'sMEnD2', 'sMED2')
sMEnD3_vs_sMED3 = compare_groups(df, 'sMEnD3', 'sMED3')
sMEnD4_vs_sMED4 = compare_groups(df, 'sMEnD4', 'sMED4')

base_folder = '/Users/bowen/Desktop/multi_omics_analysis/2408_pea 2/scripts/240825_mummichog'

folder_name = 'sMEn_vs_sMED'
folder_path = os.path.join(base_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)

file_name = 'sMEnD1_vs_sMED1.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD1_vs_sMED1.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD2_vs_sMED2.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD2_vs_sMED2.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD3_vs_sMED3.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD3_vs_sMED3.to_csv(file_path, sep='\t', index=False)

file_name = 'sMEnD4_vs_sMED4.txt'
file_path = os.path.join(folder_path, file_name)
sMEnD4_vs_sMED4.to_csv(file_path, sep='\t', index=False)